In [1]:
pip install tika

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from tika import translate
from tika import language
import requests
import re
import emoji


In [2]:
df = pd.read_csv('/Users/taniadawood/Desktop/DSCI550-PixstoryMediaExtractionAndAnalysis/1_Language Identification/Master_Dataset_Raw_LangDetect_Complete.csv')


In [3]:
df['Narrative'] = df['Narrative'].astype(str)

In [4]:
filter = df['Narrative TikaDetect'] != 'en'
df.where(filter, inplace=True)
df.dropna(inplace=True)
df = df.head(4)
df

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Date (No Timestamp),Interest,Narrative LangDetect,Narrative TikaDetect
25,88954.0,STY1647958076,2.0,USR1605614195,others,27.0,Good morning! 🙂,Hope you guys are having a great week!,https://image.pixstory.com/Pixstory-image-1647...,2020-11-17 17:26:35,2020-11-17,good morning,[en:0.9999983825560843],br
40,121502.0,STY1659507311,4.0,USR1605615013,male,33.0,This is new test,This is new test \n@ arightside \nhttps://www....,https://image.pixstory.com/Pixstory-image-1659...,2020-11-17 17:40:00,2020-11-17,"Technology, History, Food, Entertainment, Spor...","[et:0.8571383634949571, en:0.14285925329035773]",ht
42,122143.0,STY1659682079,4.0,USR1605615013,male,33.0,Hello @kumarvijay,"<p dir=""ltr"">Hello @<a href="" https://www.pixs...",https://image.pixstory.com/Pixstory-image-1659...,2020-11-17 17:40:00,2020-11-17,"Economy, Climate change , Technology, History,...","[lv:0.7142804590927756, en:0.2857140372265599]",mt
43,125240.0,STY1660311432,4.0,USR1605615013,male,33.0,This is new test,https://adachikan.com/UAT/checkout \nkumarvija...,https://image.pixstory.com/Pixstory-image-1631...,2020-11-17 17:40:00,2020-11-17,test,"[lv:0.7142821140213415, fi:0.14285728512464835...",ht


In [5]:
#taken from the emoji docs since it's not in the latest versions
def get_emoji_regexp():
    # Sort emoji by length to make sure multi-character emojis are
    # matched first
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

exp = get_emoji_regexp()

#this function removes any emojis, html tags, URLs, and special characters before translating
def clean_text(text):
    try:
        text = exp.sub(u'', text)
        text = re.sub(r'<.*?>', '', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub('[^a-zA-Z]+', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    except :
        print(text)
    finally:
        return text
    
    
clean_narr = clean_text(df['Narrative'].apply(clean_text))

df['Clean Narrative'] = clean_narr

25               Hope you guys are having a great week 
40    This is new test arightside kumarvijay kumarvi...
42                                    Hello kumarvijay 
43                               kumarvijay kumarvijay 
Name: Narrative, dtype: object


In [ ]:
chars = 20 #specifies the number of characters to translate out of the narrative

translated_column = []
for text, language in zip(df['Clean Narrative'], df['Narrative TikaDetect']):
    if language != 'en':
        if len(text) > chars:
            translated_chunks = []
            try:
                #translate the narrative in chunks depending on the limit set and join them all together at the end
                for i in range(0, len(text), chars):
                    group = text[i:i+chars]
                    translation = translate.from_buffer(group, language, 'en', requestOptions={'timeout':500})
                    if ((translation == '') or (not translation)):
                        raise ValueError('Tika server returned 500')
                    translated_chunks.append(translation)
                translated_text = ''.join(translated_chunks)
                translated_column.append(translated_text)
                
            except Exception as error:
                print(error)
                print('error text', text)
                #Appending "redo" for narratives that fail so they will be searchable to run again and update
                translated_column.append('redo')
        else:
            try:
                #translate the entire narrative if it's under the specified char size
                translated_text = translate.from_buffer(text, language, 'en', requestOptions={'timeout':500})
                if ((translated_text == '') or (not translated_text)):
                    raise ValueError('Tika server returned 500')
                translated_column.append(translated_text)
            except Exception as error:
                print(error)
                print('error text', text)
                #Appending "redo" for narratives that fail so they will be searchable to run again and update
                translated_column.append('redo')
    else:
        #just append the regular text if it's already english
        translated_column.append(text)

In [ ]:
df['Translated Narrative'] = translated_column

In [ ]:
df.to_csv('Master_Dataset_Translations_Complete', index=False)